<a href="https://colab.research.google.com/github/alexv710/debias_cv_data/blob/main/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import io

In [21]:
temp_data = pd.io.stata.read_stata("data_candidates_sample_tab6.dta")
temp_data.to_csv('data_candidates_sample_tab6.csv')

temp_data = pd.io.stata.read_stata("data_candidates_sample_tab7_tabA9.dta")
temp_data.to_csv('data_candidates_sample_tab7_tabA9.csv')

temp_data = pd.io.stata.read_stata("data_recruiters.dta")
temp_data.to_csv('data_recruiters.csv')

FileNotFoundError: ignored

In [44]:
# read the mainsample.dta file for further processing 
data = pd.io.stata.read_stata("data_candidates_mainsample.dta")
data.to_csv('data_candidates_mainsample.csv')

In [17]:
# Import the mainsample via StataReader to use variable_labels (old version of the read_stata)
data_stata = pd.io.stata.StataReader("data_candidates_mainsample.dta")
data_labels = data_stata.variable_labels()

In [23]:
data.columns

Index(['ID_OFFRE', 'ID_CANDIDAT', 'CVA', 'REFUSAL', 'ENTRETIEN', 'RECRUTE',
       'POIDS_SEL', 'PREN_MUSULMAN', 'ZUS_CUCS', 'ORIGINE_IM_1',
       ...
       'SALREV_SMIC_2_c', 'ZouI_pred', 'BAD_ANO', 'CVAxetudes_etranger',
       'CVAxtravail_etranger', 'CVAxlangue_arabe', 'CVAxBAD_ANO', 'MI',
       'CVAxMI', 'missing_CV'],
      dtype='object', length=938)

In [19]:
# Write the Labels into a csv
import csv

with open('dataLabels.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, data_labels.keys())
    w.writeheader()
    w.writerow(data_labels)

In [48]:
# Remove all columns in the dataset for which no labels exist
# Those are mostly columns needed for p-tests and some matrix calculations in
# Stata. The outputted labels are those that are kept in the data

for key, value in data_labels.items():
  for col in data.columns:
    if key==col:
      if len(value)==0 or value.startswith('ID_OFFRE=='):
        data = data.drop(columns=col)
      else:
        print(key + ':      ', value)


ID_OFFRE:       Identifier vacancy
ID_CANDIDAT:       Identifier candidate
CVA:       Treatment: anonymous resume
REFUSAL:       Recruiter refused the experiment
ENTRETIEN:       Interviewed
RECRUTE:       Hired
POIDS_SEL:       Sampling weights (within and out of the experiment)
PREN_MUSULMAN:       Muslim souding name
ZUS_CUCS:       Deprived neighborhood
ORIGINE_IM_1:       Immigrant
ORIGINE_IM_2:       Child of immigrant (father)
ORIGINE_IM_12:       Immigrant or child of immigrant
ZouI:       Minority (immigrant or child of immigrant or residing in deprived neighborhood)
ZetI:       Residing in deprived neighborhood and with foreign bachkground (child of or immi
FEMME:       Female candidate
a50p:       Candidate over 50 years old
a3049:       Candidate between 30 and 49 years old
a26m:       Candidate below 26 years old
dip_aucun:       No diploma
dip_bepcap:       Professional degree
dip_bac:       High school diploma
dip_bac2p:       Upper education degree
dip_bac2:       L2 (d

In [37]:
data['']

,ID_OFFRE,ID_CANDIDAT,CVA,REFUSAL,ENTRETIEN,RECRUTE,POIDS_SEL,PREN_MUSULMAN,ZUS_CUCS,ORIGINE_IM_1,ORIGINE_IM_2,ORIGINE_IM_12,ZouI,ZetI,FEMME,a50p,a3049,a26m,dip_aucun,dip_bepcap,dip_bac,dip_bac2p,dip_bac2,dip_bac3p,duree_expro_offre,recherche_LD,recherche_TLD,SALREV_SMIC_2,FEMME_c,a30m_c,a3049_c,a50p_c,dip_aucun_c,dip_bepcap_c,dip_bac_c,dip_bac2p_c,recherche_LD_c,recherche_TLD_c,permis,etudes_etranger,...,CV_qualif_inf,CV_experience_,CV_attrayant_,CV_formation_,effent_200plus,service_nm,service_m,industrie,construction,cadre,profint,eoq,CDI,CNT_sup6m,poste_unique,ALE_1,ALE_2,ALE_3,ALE_4,ALE_5,ALE_6,ALE_7,ALE_8,ALE_9,ALE_10,ALE_11,ALE_12,ALE_13,ALE_14,ALE_15,ALE_16,CVAxORIGINE_IM_12,CVAxZUS_CUCS,CVAxZetI,C,ZouI0,CVA0,p_offre1,control_manquant,ZouI_pred
0,1.0,1759.0,1.0,NaN,0.0,0.0,5.352863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,1.0,0.504418,0.522425,-0.411828,-0.110596,-0.055304,0.799674,-0.22715,-0.51722,-0.30711,-0.115869,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.998203,0.0,NaN
1,1.0,6177.0,1.0,NaN,0.0,0.0,5.352863,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.504418,0.522425,-0.411828,-0.110596,-0.055304,0.799674,-0.22715,-0.51722,0.69289,0.884131,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.998203,0.0,NaN
2,2.0,371.0,0.0,NaN,0.0,0.0,2.676431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.495582,-0.477575,0.588172,-0.110596,-0.055304,0.799674,-0.22715,-0.51722,0.69289,-0.115869,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,0.0,0.0,0.0,1.0,1.0,1.0,-0.001797,0.0,NaN
3,3.0,230.0,0.0,NaN,0.0,0.0,2.676431,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,4.0,0.0,0.0,0.0,-0.495582,0.522425,-0.411828,-0.110596,-0.055304,-0.200326,-0.22715,0.48278,-0.30711,-0.115869,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,0.0,0.0,0.0,1.0,0.0,1.0,-0.001797,0.0,NaN
4,4.0,1885.0,0.0,NaN,0.0,0.0,5.352863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0,0.0,-0.495582,-0.477575,0.588172,-0.110596,-0.055304,-0.200326,-0.22715,0.48278,-0.30711,-0.115869,1.0,0.0,...,0.0,4.0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,0.0,0.0,0.0,1.0,1.0,1.0,-0.001797,0.0,0.359026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,776959.0,NaN,NaN,1.0,1.0,1.0,7.238095,1.0,0.0,0.0,1.0,1.0,1.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,NaN,NaN,NaN,1.0,0.0,NaN,NaN,1.0,NaN
1467,776959.0,NaN,NaN,1.0,0.0,0.0,61.523811,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.5,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN
1468,776959.0,NaN,NaN,1.0,0.0,0.0,61.523811,NaN,0.0,1.0,0.0,1.0,1.0,NaN,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.5,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,NaN,NaN,NaN,1.0,0.0,NaN,NaN,1.0,NaN
1469,820.0,NaN,NaN,1.0,0.0,0.0,14.476191,0.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [46]:
data.to_csv('data_mainsample_cleaned.csv')